In [164]:
from bisect import bisect_left
from random import uniform
from typing import Callable
import random as rd

def cum_sum(l: list):
    ret = l[:]
    for i in range(1, len(l)):
        ret[i] += ret[i - 1]
    return ret


def gen_points(density):
    W = 1
    H = 1
    GRID_SIZE = 0.002

    line_weights = [
        cum_sum([
            density(x * GRID_SIZE, y * GRID_SIZE)
            for y in range(int(H // GRID_SIZE))
        ]) for x in range(int(W // GRID_SIZE))
    ]

    col_weight = cum_sum([line_weights[x][-1] for x in range(int(W // GRID_SIZE))])
    deja_vu = set()

    while True:
        r = uniform(0, col_weight[-1])
        x = bisect_left(col_weight, r)
        y = bisect_left(line_weights[x], uniform(0, line_weights[x][-1]))
        if not((x,y) in deja_vu):
            yield (np.array([x, y]) * GRID_SIZE)
            deja_vu.add((x,y))
            


In [165]:
import time
TIME = -1
def chrono(s):
    global TIME
    if TIME == -1:
        TIME = time.time()
    else:
        print(f"{time.time()-TIME} s elapsed for {s}")
        TIME = time.time()


In [166]:
from shapely.geometry import Polygon
def get_centroid(cell):
    return np.array(Polygon(cell).centroid.coords)[0]

In [167]:
np.array(get_centroid([[0,1], [1,1], [0,1]]))

array([0.5, 1. ])

In [180]:
import scipy
from scipy.spatial import Voronoi, voronoi_plot_2d
import numpy as np
import matplotlib.pyplot as plt


def get_relaxed_points():

    def density(x,y):
        return 1.

    points = []
    N = 1500
    MAX_ITERATION = 5 #10
    rho = 0.5 #0
    SAMPLING_MARGIN = 0.2
    MAX_MOUV = 0.002


    pts_gen = gen_points(density=density)


    for i in range(N):
        p = np.random.uniform(-SAMPLING_MARGIN, 1+SAMPLING_MARGIN, 2)

        if 0 < p[0] and p[0] < 1 and 0 < p[1] and p[1] < 1:
            points.append(next(pts_gen))
        else:
            points.append(p)

    points = np.array(points)


    for it in range(MAX_ITERATION):

        vor = Voronoi(points, qhull_options = "Qbb Qz Qc")
        
        centroids = []

        for k in range(N):
            
            if -1 in vor.regions[vor.point_region[k]]:
                center = points[k]
            else:
                cell = vor.vertices[vor.regions[vor.point_region[k]]]
                center = get_centroid(cell)
            centroids.append(center)

        ## move toward centroids
        for k in range(N):
            vect = centroids[k]-points[k]
            if np.linalg.norm(vect) > MAX_MOUV:
                points[k] = points[k] + MAX_MOUV*vect/np.linalg.norm(vect)
            else:
                points[k] = (1-rho)*points[k] + rho*centroids[k]


    vor = Voronoi(points, qhull_options = "Qbb Qz Qc")


    inner_circle_radii = []
    for k in range(N):
        p3 = centroids[k]
        reg_corners = vor.regions[vor.point_region[k]]
        rad = 100
        for corner in range(len(reg_corners)):
            p1 = vor.vertices[reg_corners[corner-1]]
            p2 = vor.vertices[reg_corners[corner]]
            #print(p1, p2, p3)
            rad = min(rad, np.linalg.norm(np.cross(p2-p1, p1-p3))/np.linalg.norm(p2-p1))
        inner_circle_radii.append(rad)
    
    
    return centroids, inner_circle_radii, vor




In [181]:


def getLine(cx, cy, angle, length, sf):
    l = length/2
    x1 = cx + np.cos(angle)*l
    y1 = cy + np.sin(angle)*l

    x2 = cx - np.cos(angle)*l
    y2 = cy - np.sin(angle)*l
    return x1, y1, x2, y2



In [182]:
COLOR_PALETTES = {
    "perso-easy" : {"background": "#01132c", "dots" : ["#fed9b7", "#ffffff", "#f07167"], "lines" : ["#ffffff", "#00afb9", "#0081a7"]},
    "perso-hard" : {"background": "#01132c", "dots" : ["#f86624", "#f9c80e", "#ea3546"], "lines" : ["#cee076", "#43bccd", "#662e9b"]},
    "about-the-world" : {"background": "#01132c", "dots" : ["#a4574d", "#43bccd", "#cee076"], "lines" : ["#a4574d", "#43bccd", "#cee076"]},
    "view-about-the-world" : {"background": "#01132c", "dots" : ["#41ead4", "#fbff12", "#ff206e"], "lines" : ["#41ead4", "#fbff12", "#ff206e"]}
}

In [186]:
import drawSvg as draw
from dataclasses import dataclass

@dataclass
class ProtectedZones:
    LOGO_XY: np.array
    LOGO_RADIUS: float
    CARD_FACE: str
    def check_not_colide(self, shape_dict, shape_type):
        if shape_type=="circle":
            p = np.array([shape_dict["cx"], shape_dict["cy"]])
            if self.CARD_FACE=="dos":
                if np.linalg.norm(self.LOGO_XY - p) < (self.LOGO_RADIUS+shape_dict["r"]):
                    return False #there is colision
                else:
                    return True # Ales gut
            if self.CARD_FACE=="ventre":
                pass
        if shape_type=="line":
            p1 = np.array([shape_dict["x1"], shape_dict["y1"]])
            p2 = np.array([shape_dict["x2"], shape_dict["y2"]])
            if self.CARD_FACE=="dos":
                if np.linalg.norm(self.LOGO_XY - p1) < self.LOGO_RADIUS or \
                   np.linalg.norm(self.LOGO_XY - p2) < self.LOGO_RADIUS :
                    return False
                else:
                    return True
            if self.CARD_FACE=="ventre":
                pass

# TODO seed the card
S = 500
SHOW_VOR = False
SHOW_DESIGN = True
shrink_factor = 0.9
PROP_CIRCLE = 0.5
LINE_WIDTH = 5

card_type = "perso-easy"
CARD_FACE = "dos" #"ventre" ou "dos"


LOGO_H = 353*0.5
LOGO_W = 273*0.5

NoMansLand = ProtectedZones(LOGO_XY = np.array([0.5, 0.5]), LOGO_RADIUS=0.2, CARD_FACE = CARD_FACE)


d = draw.Drawing(S, S, origin=(0,0), displayInline=False)

d.append(draw.Rectangle(0,0,S,S, fill=COLOR_PALETTES[card_type]['background']))



if CARD_FACE=="ventre": #font size 1000 = 1 en coo
    d.append(draw.Text(['xpldmx'], 1000, 0, 0, fill='blue', font="EB Garamond", style="font-style:normal;font-variant:normal;font-weight:normal;font-stretch:normal;font-family:'Arbutus Slab';-inkscape-font-specification:'Arbutus Slab'"))
elif CARD_FACE == "dos":
    d.append(draw.Circle(NoMansLand.LOGO_XY[0]*S, NoMansLand.LOGO_XY[1]*S, 0.01*S,
                fill="violet", stroke='none'))

    d.append(draw.Image(NoMansLand.LOGO_XY[0]*S-LOGO_W/2, 
                        NoMansLand.LOGO_XY[1]*S-LOGO_H/2, LOGO_H, LOGO_W,embed=True,
                        path=f"logo/logo-{card_type}.svg"))      

    d.append(draw.Circle(NoMansLand.LOGO_XY[0]*S-LOGO_W/2, 
                        NoMansLand.LOGO_XY[1]*S-LOGO_H/2, 0.01*S,
                fill="red", stroke='none'))


chrono("")
pts, radii, vor = get_relaxed_points()
chrono("relaxation")

for i,p in enumerate(pts):
    if SHOW_DESIGN:
        if np.random.random() < PROP_CIRCLE:
            shape_color = rd.choice(COLOR_PALETTES[card_type]['dots'])
            if NoMansLand.check_not_colide({"cx":p[0], "cy":p[1], "r":radii[i]}, "circle"):
                d.append(draw.Circle(p[0]*S, p[1]*S, radii[i]*S*shrink_factor,
                    fill=shape_color, stroke='none'))

        else:
            shape_color = rd.choice(COLOR_PALETTES[card_type]['lines'])
            angle = np.random.uniform(0, 6.28)
            x1,y1,x2, y2 = getLine(p[0], p[1], angle, radii[i], shrink_factor)
            if NoMansLand.check_not_colide({"x1": x1, "y1": y1, "x2": x2, "y2": y2}, "line"):
                d.append(draw.Line(x1*S, y1*S, x2*S, y2*S, 
                    fill='none', stroke_width=LINE_WIDTH, stroke=shape_color,stroke_linecap= "round"))

    if SHOW_VOR:
        #print(p, radii)

        d.append(draw.Circle(p[0]*S, p[1]*S, radii[i]*S,
            fill='none', stroke_width=0.5, stroke='blue'))
        d.append(draw.Circle(p[0]*S, p[1]*S, 1.,
                fill='black'))
        reg_corners = vor.regions[vor.point_region[i]]
        for corner in range(len(reg_corners)):
            if reg_corners[corner-1] != -1 and reg_corners[corner] != -1:
                p1 = vor.vertices[reg_corners[corner-1]]
                p2 = vor.vertices[reg_corners[corner]]
                d.append(draw.Line(p1[0]*S, p1[1]*S, p2[0]*S, p2[1]*S,
                    stroke='red', stroke_width=0.5, fill='none'))
        

d.setPixelScale(2)  # Set number of pixels per geometry unit
d.saveSvg('example.svg')
d.rasterize()  # Display as PNG
chrono("drawing")
d


10.253623962402344 s elapsed for 
1.0181317329406738 s elapsed for relaxation
0.3360567092895508 s elapsed for drawing


In [ ]:
1000 pt = 400 d'unit svg

In [ ]:
draw.TSpan()

In [118]:
for k in range(10):
    cell = vor.vertices[vor.regions[vor.point_region[k]]]
    center = get_centroid(cell)
    print(vor.regions[vor.point_region[k]])

[684, 677, 678, 487, 486, 485, 683]
[693, 569, 567, 691]
[870, 714, 715, 713, 792, 868]
[629, 387, 386, 385, 384, 396, 628]
[834, 830, 453, 235, 237, 236, 831]
[645, 441, 439, 217, 223, 222, 216, 435, 643]
[223, 217, 213, 215, 214, 221]
[510, 498, 495, 282, 281, 508]
[315, 313, 312, 311, 314]
[950, 900, 878, 877, 879, 880, 949]


In [119]:
np.sin(3.141592/2)

0.9999999999999466